In [1]:
import xarray
import spires
import numpy as np
import matplotlib.pyplot as plt

In [2]:
interpolator = spires.LutInterpolator(lut_file='../tests/data/lut_sentinel2b_b2to12_3um_dust.mat')
bands = interpolator.bands
solar_angles = interpolator.solar_angles
dust_concentrations = interpolator.dust_concentrations
grain_sizes = interpolator.grain_sizes
reflectances = interpolator.reflectances

In [3]:
r = xarray.load_dataset('../tests/data/sentinel_r.nc')
r0 = xarray.load_dataset('../tests/data/sentinel_r0.nc')
x0 = np.array([0.5, 0.05, 10, 250])

In [4]:
r

In [5]:
date = '2024-02-25'
ts = r.sel(time=date).squeeze().drop_vars('time')

In [8]:
spectra_targets = ts['reflectance'].transpose('y', 'x', 'band')
spectra_backgrounds = r0['reflectance'].transpose('y', 'x', 'band')
obs_solar_angles = ts['sun_zenith_grid'].transpose('y', 'x')
spectrum_shade = np.zeros_like(spectra_targets[0,0,:])
spectra_targets.shape
results = np.empty((spectra_targets.y.size, spectra_targets.x.size, 4), dtype=np.double)

In [30]:
np.zeros(spectra_targets.band.size, dtype=np.double)

In [26]:
spectrum_shade

In [25]:
spectra_targets.y.size
spectra_targets.x.size

In [9]:
spectrum_shade

In [10]:
spectra_targets.values[0, 0,:]

In [11]:
spectra_targets.values.flatten()[0:9]

In [12]:
spires.core.invert_array2d(spectra_backgrounds=spectra_backgrounds, 
                           spectra_targets=spectra_targets, 
                           spectrum_shade=spectrum_shade, 
                           obs_solar_angles=obs_solar_angles, 
                           bands=bands, solar_angles=solar_angles, dust_concentrations=dust_concentrations, grain_sizes=grain_sizes, lut=reflectances, 
                           results=results, 
                           max_eval=100, 
                           x0=x0, 
                           algorithm=2)

In [17]:
results[:,:,1].max()

In [21]:
plt.imshow(results[:,:, 2])

In [ ]:
%%time
results = spires.speedy_invert_array1d(spectra_targets=spectra_targets.values,
                                       spectra_backgrounds=spectra_backgrounds.values,
                                       obs_solar_angles=obs_solar_angles.values,
                                       interpolator=interpolator)

### 